In [168]:
# importing the requests library
import requests
import json
import numpy as np
from functools import reduce

In [210]:
def get_result(url):
    # defining a params dict for the parameters to be sent to the API
    header = { 'X-API-KEY':"gXhJxaWsNLUTfL9UeGwak4hf1F8fcunI7AR7HfCV"}
    # sending get request and saving the response as response object
    r = requests.get(url = url, headers = header)
    # extracting data in json format
    data = r.json()
    return data['results']

def generate_urls_recent_votes(number):
    urls = []
    offset = 0
    for i in range(number):
        offest = i * 20
        url = "https://api.propublica.org/congress/v1/house/votes/recent.json?offset=" + str(offset)
        urls.append(url)
    return np.array(urls)

In [361]:
urls = generate_urls_recent_votes(20) # 400 BILLS
recent_votes_results = [get_result(url) for url in urls]
recent_votes_results_flat = [vote for votes in recent_votes_results for vote in votes['votes']]

In [366]:
# BILLS INFORMATION TABLE

def get_bill(vote):
    bill_uri = vote['bill']['api_uri']
    if bill_uri is None:
        return None
    return get_result(bill_uri)[0]

def make_bill_row(vote):
    bill_id = vote['bill']['bill_id']
    summary = get_bill_summary_from_vote(vote)
    date = vote['date']
    return pd.DataFrame([[bill_id, summary, date]], columns=cols)

In [367]:
# PROCESS VOTES

def process_vote(vote):    
    bill = get_bill(vote)
    if bill is None:
        return None, None
    
    bill_slug = bill['bill_slug']
    bill_summary = bill['summary']
    bill_date = vote['date']
    bill_type = bill['bill_type']
    congress = bill['congress']
    bill_uri = bill['bill_uri']
    
    cols = ['bill_slug', 'summary', 'date', 'bill_type', 'congress', 'bill_uri']
    values = [[bill_slug, bill_summary,bill_date,bill_type, congress, bill_uri]]
    bill_row_df = pd.DataFrame(values, columns=cols)
    
    actual_vote = get_result(vote['vote_uri'])
    positions = actual_vote['votes']['vote']['positions']
    vote_df = pd.DataFrame(positions)
    vote_df = vote_df.rename(columns={'vote_position': bill_slug}).drop('dw_nominate', 1)
    return vote_df, bill_row_df

In [368]:
# HACK TO CREATE DUMMY TABLE - SKETCH
vote_df, bill_row = process_vote(recent_votes_results_flat[0])
vote_df.drop(vote_df.columns[len(vote_df.columns)-1], axis=1, inplace=True)
vote_df 

,district,member_id,name,party,state
0,5,A000374,Ralph Abraham,R,LA
1,12,A000370,Alma Adams,D,NC
2,4,A000055,Robert B. Aderholt,R,AL
3,31,A000371,Pete Aguilar,D,CA
4,12,A000372,Rick Allen,R,GA
5,3,A000367,Justin Amash,R,MI
6,2,A000369,Mark Amodei,R,NV
7,19,A000375,Jodey Arrington,R,TX
8,36,B001291,Brian Babin,R,TX
9,2,B001298,Don Bacon,R,NE


In [369]:
votes_df = vote_df
bills_df = pd.DataFrame()
count = 0
for vote in recent_votes_results_flat:
    vote_df, bill_row = process_vote(vote)
    if vote_df is None or bill_row is None:
        count += 1
        print("Found {} nulls".format(count))
        continue
    votes_df = pd.merge(votes_df, vote_df, on=['member_id', 'name', 'district', 'state', 'party'], how='outer')
    bills_df = bills_df.append(bill_row)

Found 1 nulls
Found 2 nulls
Found 3 nulls
Found 4 nulls
Found 5 nulls
Found 6 nulls
Found 7 nulls
Found 8 nulls
Found 9 nulls
Found 10 nulls
Found 11 nulls
Found 12 nulls
Found 13 nulls
Found 14 nulls
Found 15 nulls
Found 16 nulls
Found 17 nulls
Found 18 nulls
Found 19 nulls
Found 20 nulls
Found 21 nulls
Found 22 nulls
Found 23 nulls
Found 24 nulls
Found 25 nulls
Found 26 nulls
Found 27 nulls
Found 28 nulls
Found 29 nulls
Found 30 nulls
Found 31 nulls
Found 32 nulls
Found 33 nulls
Found 34 nulls
Found 35 nulls
Found 36 nulls
Found 37 nulls
Found 38 nulls
Found 39 nulls
Found 40 nulls


In [370]:
votes_df

,district,member_id,name,party,state,hr1_x,hr2672_x,hr3821_x,hr2331_x,hres619_x,...,hr2874_y,hres616_x,hres616_y,hr3071_y,hres599_y,hr2201_y,hr3705_y,hr3043_x,hr3043_y,hr4173_y
0,5,A000374,Ralph Abraham,R,LA,Yes,Yes,Yes,Yes,Yes,...,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes
1,12,A000370,Alma Adams,D,NC,No,Yes,Yes,Yes,No,...,Yes,No,No,Yes,Yes,No,Yes,No,Yes,Yes
2,4,A000055,Robert B. Aderholt,R,AL,Yes,Yes,Yes,Yes,Yes,...,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes
3,31,A000371,Pete Aguilar,D,CA,No,Yes,Yes,Yes,No,...,Yes,No,No,Yes,Yes,No,Yes,No,Yes,Yes
4,12,A000372,Rick Allen,R,GA,Yes,Yes,Yes,Yes,Yes,...,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Not Voting
5,3,A000367,Justin Amash,R,MI,Yes,Yes,Yes,Yes,Yes,...,No,No,Yes,Yes,No,Yes,Yes,Yes,No,Yes
6,2,A000369,Mark Amodei,R,NV,Yes,Yes,Yes,Yes,Yes,...,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes
7,19,A000375,Jodey Arrington,R,TX,Yes,Yes,Yes,Yes,Yes,...,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes
8,36,B001291,Brian Babin,R,TX,Yes,Yes,Yes,Yes,Yes,...,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes
9,2,B001298,Don Bacon,R,NE,Yes,Yes,Yes,Yes,Yes,...,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes


In [ ]:
bills_df

In [317]:
# FROM HERE ON ONLY TRASH

ob = get_result(recent_votes_results_flat[0]['bill']['api_uri'])

In [328]:
ob[0].keys()

dict_keys(['bill_id', 'bill_slug', 'congress', 'bill', 'bill_type', 'number', 'bill_uri', 'title', 'short_title', 'sponsor_title', 'sponsor', 'sponsor_id', 'sponsor_uri', 'sponsor_party', 'sponsor_state', 'gpo_pdf_uri', 'congressdotgov_url', 'govtrack_url', 'introduced_date', 'active', 'last_vote', 'house_passage', 'senate_passage', 'enacted', 'vetoed', 'cosponsors', 'cosponsors_by_party', 'withdrawn_cosponsors', 'primary_subject', 'committees', 'committee_codes', 'subcommittee_codes', 'latest_major_action_date', 'latest_major_action', 'house_passage_vote', 'senate_passage_vote', 'summary', 'summary_short', 'versions', 'actions', 'votes'])

In [230]:
# REPRESENTATIVES VS VOTES TABLE

def get_vote_table(vote):
    actual_vote = get_result(vote['vote_uri'])
    positions = actual_vote['votes']['vote']['positions']
    bill_id = actual_vote['votes']['vote']['bill']['bill_id']
    vote_df = pd.DataFrame(positions)
    vote_df = vote_df.rename(columns={'vote_position': bill_id}).drop('dw_nominate', 1)
    return vote_df

frames = [get_vote_table(vote) for vote in recent_votes_results_flat]

In [239]:
frames[7]

,district,member_id,name,party,state,hr2874-115
0,5,A000374,Ralph Abraham,R,LA,No
1,12,A000370,Alma Adams,D,NC,No
2,4,A000055,Robert B. Aderholt,R,AL,Yes
3,31,A000371,Pete Aguilar,D,CA,No
4,12,A000372,Rick Allen,R,GA,Yes
5,3,A000367,Justin Amash,R,MI,No
6,2,A000369,Mark Amodei,R,NV,Yes
7,19,A000375,Jodey Arrington,R,TX,Yes
8,36,B001291,Brian Babin,R,TX,Yes
9,2,B001298,Don Bacon,R,NE,Yes


In [242]:
def merge_votes(votes_table_list):
    return reduce(lambda left,right: pd.merge(left,right,on=['member_id', 'name', 'district', 'state', 'party'], how='outer'), votes_table_list)

final_df = merge_votes(frames)
final_df.to_csv('./initial_data.csv')

In [253]:
bill_slugs = [bill['bill_slug'] for bill in bills]
bill_slugs
bills

[{'active': False,
  'bill_id': 'hr5858-113',
  'bill_slug': 'hr5858',
  'bill_type': 'hr',
  'bill_uri': 'https://api.propublica.org/congress/v1/113/bills/hr5858.json',
  'committee_codes': ['HSJU'],
  'committees': 'House Judiciary Committee',
  'congressdotgov_url': 'https://www.congress.gov/bill/113th-congress/house-bill/5858',
  'cosponsors': 0,
  'cosponsors_by_party': {},
  'enacted': None,
  'govtrack_url': 'https://www.govtrack.us/congress/bills/113/hr5858',
  'gpo_pdf_uri': None,
  'house_passage': None,
  'introduced_date': '2014-12-11',
  'last_vote': None,
  'latest_major_action': 'Referred to the House Committee on the Judiciary.',
  'latest_major_action_date': '2014-12-11',
  'number': 'H.R.5858',
  'primary_subject': 'Crime and Law Enforcement',
  'senate_passage': None,
  'short_title': 'Build TRUST Act of 2014',
  'sponsor_id': 'J000032',
  'sponsor_name': 'Sheila Jackson Lee',
  'sponsor_party': 'D',
  'sponsor_state': 'TX',
  'sponsor_title': 'Rep.',
  'sponsor_uri'